In [ ]:
import pandas as pd
import glob

print(" Faz 1-2-3 Birleştiriliyor...")


# 1. AYARLAR: ARANACAK TÜM KRİTİK SATIRLAR (GELİR + RİSK)

# Dosyalardaki GERÇEK, harfi harfine isimler:
hedef_kalemler = [
    # FAZ 1 (SONUÇ) 
    "DÖNEM NET KARI (ZARARI) (51-52)",
    
    # FAZ 2 (GELİRİN KAYNAĞI) 
    "NET FAİZ (KAR PAYI) GELİRİ (GİDERİ) (15-23)",
    "Toplam Faiz (Kar Payı) Gelirleri (1+...+14)-(2+3+4+5)",
    "Toplam Faiz (Kar Payı) Giderleri (16+...+22)",
    
    # FAZ 3 (RİSK / KARŞILIKLAR) 
    "Genel Karşılık Provizyonu",  # Ana Risk (Krediler)
    "Diğer Provizyonlar"          # Yan Riskler
]

# Çıktı dosyasında sütunlar nasıl görünsün? 
isim_duzeltme = {
    "DÖNEM NET KARI (ZARARI) (51-52)": "Net_Kar",
    "NET FAİZ (KAR PAYI) GELİRİ (GİDERİ) (15-23)": "Net_Faiz_Marji",
    "Toplam Faiz (Kar Payı) Gelirleri (1+...+14)-(2+3+4+5)": "Faiz_Geliri",
    "Toplam Faiz (Kar Payı) Giderleri (16+...+22)": "Faiz_Gideri",
    "Genel Karşılık Provizyonu": "Kredi_Risk_Karsiligi",
    "Diğer Provizyonlar": "Diger_Riskler"
}


# 2. DOSYALARI OKU VE FİLTRELE

files = sorted(glob.glob("Kar Zarar_*.xlsx"))

if not files:
    print("❌ HATA: Klasörde 'Kar Zarar_*.xlsx' dosyaları bulunamadı!")
else:
    print(f"📂 {len(files)} adet dosya işleniyor...")
    
    tum_veriler = []
    
    for f in files:
        try:
            # Excel'i oku
            df = pd.read_excel(f)
            
            # "Dönem" sütununu bul ve adını standartlaştır
            period_col = [c for c in df.columns if "Dönem:" in str(c)][0]
            df = df.rename(columns={period_col: "kalem"})
            
            # Tarihi dosya isminden çıkar (Örn: Kar Zarar_2023_01.xlsx)
            parts = f.split("_")
            year = int(parts[1])
            month = int(parts[2].replace(".xlsx",""))
            tarih = pd.to_datetime(f"{year}-{month:02d}-01")
            
            # SADECE hedef kalemleri al
            df_filtered = df[df["kalem"].isin(hedef_kalemler)].copy()
            df_filtered["tarih"] = tarih
            
            tum_veriler.append(df_filtered)
            
        except Exception as e:
            print(f"⚠️ Dosya atlandı: {f} -> {e}")

  
    # 3. PİVOT VE MATEMATİK (KÜMÜLATİF -> AYLIK)
   
    if tum_veriler:
        # Alt alta birleştir
        full_df = pd.concat(tum_veriler, ignore_index=True)
        
        # PIVOT: Satırlar Tarih, Sütunlar Kalemler olsun
        df_pivot = full_df.pivot_table(
            index="tarih", 
            columns="kalem", 
            values="Toplam", 
            aggfunc="sum"
        ).sort_index()
        
        # Matematik: Bu ayın değerinden geçen ayı çıkar (Diff)
        # (Bankacılık verisi kümülatif gelir, biz aylık neti buluyoruz)
        df_master = df_pivot.groupby(df_pivot.index.year).diff()
        
        # Ocak aylarını düzelt (Ocak ayının farkı kendisidir, NaN olmasın)
        df_master = df_master.fillna(df_pivot)
        
       
        # 4. SON RÖTUŞLAR VE KAYIT
      
        # İsimleri değiştir
        df_master = df_master.rename(columns=isim_duzeltme)
        
        # Tarihi sütuna geri al
        df_master = df_master.reset_index()
        
        # Kaydet
        dosya_adi = "Proje_Final_Master_Data.xlsx"
        df_master.to_excel(dosya_adi, index=False)
        
        print("\n İŞLEM TAMAMLANDI! ")
        print(f" Dosyanız Hazır: {dosya_adi}")
        print("-" * 50)
        print("İşte tablonun son hali (İlk 5 Satır):")
        print(df_master.head())
        
    else:
        print(" Hiç veri oluşturulamadı. Dosyaları kontrol et.")

🚀 BÜYÜK FİNAL: Faz 1-2-3 Birleştiriliyor...
📂 35 adet dosya işleniyor...


c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains


✅ İŞLEM TAMAMLANDI! 🏆
📁 Dosyanız Hazır: Proje_Final_Master_Data.xlsx
--------------------------------------------------
İşte tablonun son hali (İlk 5 Satır):
kalem      tarih  Diger_Riskler    Net_Kar  Kredi_Risk_Karsiligi  \
0     2023-01-01       2995.365  39319.844             19307.855   
1     2023-02-01       2441.452  26243.944             14453.106   
2     2023-03-01       5443.215  40530.993             14224.731   
3     2023-04-01       1343.330  38604.425              5867.626   
4     2023-05-01       3072.636  45645.049             10354.702   

kalem  Net_Faiz_Marji  Faiz_Geliri  Faiz_Gideri  
0           52209.314   133265.235    81055.921  
1           42988.944   123174.141    80185.197  
2           52250.895   150428.535    98177.640  
3           42234.819   145804.501   103569.682  
4           34168.093   156065.158   121897.065  


c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\ProgramData\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
